In [4]:
from os import listdir
from os.path import isfile, join

In [6]:
def load_data_tales(ratio):

    paths = ["../data/Grimms/emmood/", 
             "..data/Potter/emmood/", 
             "../data/HCAndersen/emmood/"]

    train_filenames = []
    test_filenames = []
#     ratio = 0.15
    for p in paths:
        onlyfiles = [p+f for f in listdir(p) if isfile(join(p, f))]
        
        local_ratio = int(ratio*len(onlyfiles))

        train_filenames += onlyfiles[:-local_ratio]
        test_filenames += onlyfiles[len(onlyfiles)-local_ratio:]

    examples = []
    raw_sents = []
    chapter_indices = []
    
#     for i in range(len(test_filenames)):
#         test_filenames[i] = test_filenames[i].replace("/Users/chenyanglyu/PycharmProjects/text2story/", "")
    
#     print(test_filenames)
    
    c_index = 0
    for data_dir in train_filenames:
        with open(data_dir, "r") as f:
            lines = f.readlines()

            for (i, line) in enumerate(lines):
                guid = "%s-%s" % ("fairy_tales", i)
                a, b, c, d = line.split('\t')
                text_a = d[:-1]
                label = b.split(':')[0]

                chapter_indices.append(c_index)

                raw_sents.append(text_a)
                examples.append(label)
            c_index += 1


    test_examples = []
    test_raw_sents = []
    test_chapter_indices = []

    c_index = 0
    for data_dir in test_filenames:
        with open(data_dir, "r") as f:
            lines = f.readlines()

            for (i, line) in enumerate(lines):
                guid = "%s-%s" % ("fairy_tales", i)
                a, b, c, d = line.split('\t')
                text_a = d[:-1]
                label = b.split(':')[0]

                test_chapter_indices.append(c_index)

                test_raw_sents.append(text_a)
                test_examples.append(label)
            c_index += 1


    return raw_sents, examples, test_raw_sents, test_examples, chapter_indices, test_chapter_indices

for ratio in range(10,11):
    ratio /= 100
    
    _, t_labels, __, test_labels, *_= load_data_tales(ratio)
    
    se = {}

    for t in test_labels:
        if t in se:
            se[t] += 1
        else:
            se[t] = 1
            
    total = 0

    data = se

    for k in data:
        total += data[k]
    
    print(se,se['N']/total, total, len(t_labels), len(test_labels))
    

{'N': 688, 'Su+': 30, 'H': 117, 'Sa': 82, 'A': 121, 'D': 28, 'Su-': 53, 'F': 70} 0.5786375105130361 1189 14113 1189


In [7]:
label_list = ["A", "D", "F", "H", "N", "Sa", "Su+", "Su-"]
pos_list = ["H", "Su+"]
neg_list = ["A", "D", "F", "Sa", "Su-"]
binary_label_list = ["N", "E"]

train_sents, train_labels, test_sents, test_labels, train_cs, test_cs = load_data_tales(0.1)

total_sents = train_sents + test_sents
total_labels = train_labels + test_labels
total_cs = train_cs + test_cs
class Lang:
    # statistic for natural sentence, including word to index, word count,etc
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0  # Count SOS and EOS
        self.words_count = 0

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        self.words_count += 1
            
    
total_lang = Lang('total')
for sent in total_sents:
    total_lang.addSentence(sent)

In [8]:
langs = []
n_lang = Lang('n')
e_lang = Lang('e')
b_langs = [n_lang, e_lang]
binary_labels = []

for l in total_labels:
    _l = "N" if l == "N" else "E"
    binary_labels.append(_l)

for i in range(8):
    lang = Lang(i)
    langs.append(lang)

for label,sent in zip(binary_labels, total_sents):
    b_langs[binary_label_list.index(label)].addSentence(sent)
    
for label,sent in zip(total_labels, total_sents):
    langs[label_list.index(label)].addSentence(sent)

In [19]:
words_fre = []
thre = 600
low_thre = 5
for w in total_lang.word2count:
    if total_lang.word2count[w] > thre or total_lang.word2count[w] < low_thre:
        continue
    for i in range(8):
        sta = {}
        sta['word index'] = total_lang.word2index[w]
        sta['frequency'] = langs[i].word2count[w] if w in langs[i].word2count else 0
        sta['normalized frequency'] = sta['frequency']/langs[i].words_count
        sta['class'] = label_list[i]
        sta['word'] = w
        words_fre.append(sta)
        
b_words_fre = []
for w in total_lang.word2count:
    if total_lang.word2count[w] > thre or total_lang.word2count[w] < low_thre:
        continue
    for i in range(2):
        sta = {}
        sta['word index'] = total_lang.word2index[w]
        sta['frequency'] = b_langs[i].word2count[w] if w in b_langs[i].word2count else 0
        sta['normalized frequency'] = sta['frequency']/langs[i].words_count
        sta['class'] = binary_label_list[i]
        sta['word'] = w
        b_words_fre.append(sta)

In [30]:
import plotly.express as px
fig = px.scatter(b_words_fre, x="word index", y="normalized frequency", color="class")
fig.show()

In [60]:
import plotly.express as px
fig = px.scatter(words_fre, x="word index", y="normalized frequency", color="class")
fig.show()

In [32]:
markov_matrix = [[0 for i in range(8)] for j in range(8)]
binary_m_matrix = [[0 for i in range(2)] for j in range(2)]
last_ci = -1
last_label = -1
for ci, label in zip(total_cs, total_labels):
    if last_ci == -1:
        last_ci = ci
        last_label = label
        continue
    if ci == last_ci:
        i = label_list.index(last_label)
        j = label_list.index(label)
        markov_matrix[i][j] += 1
        
    last_ci = ci
    last_label = label

last_ci = -1
last_label = -1
for ci, label in zip(total_cs, binary_labels):
    if last_ci == -1:
        last_ci = ci
        last_label = label
        continue
    if ci == last_ci:
        i = binary_label_list.index(last_label)
        j = binary_label_list.index(label)
        binary_m_matrix[i][j] += 1
        
    last_ci = ci
    last_label = label
        
        

In [33]:
markov_matrix

[[232, 20, 71, 37, 270, 54, 8, 30],
 [28, 103, 13, 20, 262, 19, 8, 9],
 [63, 12, 168, 54, 299, 53, 16, 27],
 [45, 31, 55, 443, 832, 73, 36, 41],
 [274, 251, 292, 870, 7514, 372, 195, 276],
 [42, 16, 44, 60, 411, 215, 17, 18],
 [7, 3, 19, 82, 152, 8, 41, 12],
 [38, 25, 34, 37, 247, 26, 8, 88]]

In [34]:
binary_m_matrix

[[7514, 2530], [2473, 2609]]

In [165]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import numpy as np
corpus = ['this is the first document','this document is the second document', 
          'and this is the third one','is this the first document']
vocabulary = ['this', 'document', 'first', 'is', 'second', 'the','and', 'one']
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()

a = pipe.transform(corpus)
b = [1,1,0,0]

In [166]:
train_sents, train_labels, test_sents, test_labels = load_data_tales(0.1)

corpus = train_sents + test_sents

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vocabulary = vectorizer.get_feature_names()
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),('tfid', TfidfTransformer())]).fit(corpus)
pipe['count'].transform(corpus).toarray()

a = pipe.transform(corpus)

In [167]:
train_set = a[:len(train_labels)]
test_set = a[len(train_labels):]

In [242]:
import sklearn.metrics as metric

def metrics(preds, labels):
    _test_labels = labels
    accuracy = metric.accuracy_score(_test_labels, preds)
    precision = metric.precision_score(_test_labels, preds, average='weighted')
    recall = metric.recall_score(_test_labels, preds, average='weighted')
    f1 = metric.f1_score(_test_labels, preds, average='weighted')

    print("accuracy: ", accuracy)
    print("precision: ", precision)
    print("recall: ", recall)
    print("F1: ", f1)

In [1]:
import pandas as pd

df = pd.read_csv("/Users/chenyanglyu/PycharmProjects/text2story/data/knowledge_ibm/idiomLexicon.tsv", sep="\t")

In [2]:
df.head()

,Idiom,WiktionaryURL,Pos,Neg,Neu,Inapprop.,Total,%Pos,%Neg,%Neu,Maj. Label,FilterOut(X)
0,American Dream,https://en.wiktionary.org/wiki/American_Dream,8,0,2,0,10,0.8,0.0,0.2,positive,NaN
1,Catch-22,https://en.wiktionary.org/wiki/Catch-22,0,7,3,0,10,0.0,0.7,0.3,negative,NaN
2,Christmas present,https://en.wiktionary.org/wiki/Christmas_present,6,0,4,0,10,0.6,0.0,0.4,positive,NaN
3,Downing Street,https://en.wiktionary.org/wiki/Downing_Street,0,0,10,0,10,0.0,0.0,1.0,neutral,NaN
4,Dutch courage,https://en.wiktionary.org/wiki/Dutch_courage,2,2,6,0,10,0.2,0.2,0.6,neutral,NaN
